#### ID 10351

```Find the email activity rank for each user. Email activity rank is defined by the total number of emails sent. The user with the highest number of emails sent will have a rank of 1, and so on. Output the user, total emails, and their activity rank. Order records by the total emails in descending order. Sort users with the same number of emails in alphabetical order. In your rankings, return a unique value (i.e., a unique rank) even if multiple users have the same number of emails. For tie breaker use alphabetical order of the user usernames.```

In [ ]:
%%sql
SELECT from_user,
       COUNT(from_user)                                                  AS total_emails,
       ROW_NUMBER() OVER (ORDER BY COUNT(from_user) DESC, from_user ASC) AS rnk
FROM google_gmail_emails
GROUP BY from_user
ORDER BY total_emails DESC

In [ ]:
df = google_gmail_emails
result = df.groupby('from_user', as_index=False).agg(total_emails=('from_user', 'count'))
result['rnk'] = result.sort_values(['total_emails', 'from_user'], ascending=[False, True])['total_emails'].rank(method='first', ascending=False)
result.sort_values('rnk')

#### ID 10354

```Find the 3 most profitable companies in the entire world. Output the result along with the corresponding company name. Sort the result based on profits in descending order.```

In [ ]:
%%sql
WITH ranked AS (SELECT company,
                    profits,
                    DENSE_RANK() OVER (ORDER BY profits DESC) AS rnk
             FROM forbes_global_2010_2014)
SELECT company, profits
FROM ranked
WHERE rnk <= 3
ORDER BY profits DESC

In [ ]:
df = forbes_global_2010_2014
df['rnk'] = df['profits'].rank(method='first', ascending=False)
df.query('rnk <= 3').sort_values('profits', ascending=False)[['company', 'profits']]

#### ID 10356

```Find doctors with the last name of 'Johnson' in the employee list. The output should contain both their first and last names.```

In [ ]:
%%sql
SELECT el.first_name, el.last_name
FROM employee_list el
WHERE last_name = 'Johnson'
  AND profession = 'Doctor'

In [ ]:
df = employee_list
df.query('profession == "Doctor" and last_name == "Johnson"').filter(items=['first_name', 'last_name'])